# <b>Kumho Compiler</b>

### The purpose of this compiler is to:
1. Convert the DAT files into a CSV file
2. Combine multiple CSV files into one dataframe
3. Data visualization for analysis

 <br>

### 1. Converting .DAT files into .CSV file (dattocsv.py)

#### The conversion program written in python executes the following:
        -Converts the DAT files into a CSV file.
        -Saves the CSV file in a CSV folder in the parent directory

#### Requirement:
        Run the following program in a windows machine as it runs subprocess via command line switch.
https://github.com/alpacapetter/kumho-compiler/blob/main/dattocsv.py

 <br>

### 2.  Multiple CSV files into one dataframe

In [1]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

Pressure sensor records the pressure value every 10 seconds.<br>
A raw csv file which contains 1 hour of sensor data looks like the following:

In [2]:
files = [file for file in os.listdir("./csv_data/BM1 SCRUBBER SC-101-PT-101 DATA_csv/")]
df = pd.read_csv("./csv_data/BM1 SCRUBBER SC-101-PT-101 DATA_csv/"+files[0])
df

,;Date,Time,Millitm,Tagname,Value,Status,Marker,Internal
04/06/2017,14:00:01,179,[HKT]BM1_HMI_AI16 ...,-55.560013,,B,-1,NaN
04/06/2017,14:00:06,668,[HKT]BM1_HMI_AI16 ...,-55.560013,,,0,NaN
04/06/2017,14:00:16,679,[HKT]BM1_HMI_AI16 ...,-54.779999,,,1,NaN
04/06/2017,14:00:26,689,[HKT]BM1_HMI_AI16 ...,-55.080002,,,2,NaN
04/06/2017,14:00:36,700,[HKT]BM1_HMI_AI16 ...,-55.020004,,,3,NaN
...,...,...,...,...,...,...,...,...
04/06/2017,14:59:26,680,[HKT]BM1_HMI_AI16 ...,-54.720001,,,356,NaN
04/06/2017,14:59:36,690,[HKT]BM1_HMI_AI16 ...,-55.560013,,,357,NaN
04/06/2017,14:59:46,701,[HKT]BM1_HMI_AI16 ...,-54.840012,,,358,NaN
04/06/2017,14:59:56,656,[HKT]BM1_HMI_AI16 ...,-54.779999,,,359,NaN


Average pressure value from 2pm to 3pm everyday was concatenated into a single DataFrame.

In [3]:
def date_value_cleanup(target_folder):

    files = [file for file in os.listdir("./csv_data/" + target_folder + "/")]
    new_df = pd.DataFrame(columns = ["date","val"])
    
    #for each files in the directory
    for file in files:
        # reassign column names
        df = pd.read_csv("./csv_data/" + target_folder + "/" + file,
                         names =['date', 'Del1', 'Del2', 'Del3', 'val', 'Del4', 'Del5', 'Del6']
                        )

        # take an average of the "value" column for the csv file
        mod_df = df.drop(index = 0, columns = ['Del1', 'Del2', 'Del3', 'Del4', 'Del5', 'Del6'])
        avg_val = mod_df["val"].astype(float).mean()
        avg_val = abs(avg_val)
        if avg_val == 0:
            continue

        # find date
        try:
            date = df.loc[1, "date"]
        except KeyError:
            continue

        # append to new_df
        df2 = pd.DataFrame([[date, avg_val]], columns=["date","val"])
        new_df = new_df.append(df2,ignore_index=True)

    return new_df

In [15]:
sc1_pt = date_value_cleanup("BM1 SCRUBBER SC-101-PT-101 DATA_csv")
bhd1_pt = date_value_cleanup("BM1 DROP DOOR PT-101 DATA_csv")
bhh1_pt = date_value_cleanup("BM1 HOPPER DOOR PT-101 DATA_csv")

Cleaned DataFrame with only date and value looks like this:

In [16]:
sc1_pt

,date,val
0,04/06/2017,55.424923
1,04/07/2017,51.177345
2,04/08/2017,51.059113
3,04/09/2017,50.587013
4,04/10/2017,50.994472
...,...,...
487,09/06/2018,57.919886
488,09/07/2018,56.659113
489,09/08/2018,58.253311
490,09/09/2018,59.006626


 <br>

### 3. Data visualization for analysis

In [17]:
fig = px.line(sc1_pt, x="date", y="val", title='Scrubber 1 Pressure Sensor Data')
fig.show()

By assessing scrubber1 pressure drop pattern, 3 month maintenance cycle was recommended.

<br>

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=sc1_pt["date"], y=sc1_pt["val"], name='SC1 PT',
                         line=dict(color='firebrick', width=1)))
fig.add_trace(go.Scatter(x=bhd1_pt["date"], y=bhd1_pt["val"], name = 'B/H-DD PT',
                         line=dict(color='royalblue', width=1)))
fig.add_trace(go.Scatter(x=bhh1_pt["date"], y=bhh1_pt["val"], name = 'B/H-HD PT',
                         line=dict(color='LightSeaGreen', width=1)))

# Layout Edit
fig.update_layout(title='BM1 System Pressure Sensor Data',
                   yaxis_title='Pressure')

fig.show()

As expected, both the baghouses and the scrubber pressures share similar pattern since the system is connected.

 <br>

# Let's save the clean sc1_pt data frame to SQL for easier future analysis

In [19]:
import sqlite3

In [20]:
dbconn = sqlite3.connect('mykm')

In [21]:
sc1_pt.to_sql('sc1_pt_SQL', con=dbconn, if_exists='replace')

In [22]:
pd.read_sql_query('select * from sc1_pt_SQL limit 10', con=dbconn)

,index,date,val
0,0,04/06/2017,55.424923
1,1,04/07/2017,51.177345
2,2,04/08/2017,51.059113
3,3,04/09/2017,50.587013
4,4,04/10/2017,50.994472
5,5,04/11/2017,50.543091
6,6,04/12/2017,0.808829
7,7,04/13/2017,50.650826
8,8,04/14/2017,50.307732
9,9,04/15/2017,50.758008
